<a href="https://colab.research.google.com/github/Tbharvesh/DeepLearning/blob/main/Intend_Detection_LSTM_Bank_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement : To Perform the intend detection using **LSTM** on a Bank Dataset

### Workflow :

1. Understand problem
2. Read data and ubderstand it
3. preprocess the data
4. LSTM Model
5. Training and Evaluation of model




In [1]:
import numpy as np
import pandas as pd
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

2024-04-07 16:08:08.629701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 16:08:08.629747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 16:08:08.635908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 16:08:08.675209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 16:08:12.776736: W tensorflow/compiler/tf2

True

In [121]:
pip install --upgrade ipykernel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 23.2.1
    Uninstalling pyzmq-23.2.1:
      Successfully uninstalled pyzmq-23.2.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.4 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.2 which is incompatible.


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

In [2]:
train= pd.read_csv('/content/bank_train.csv',
                       names= [ "text","target"])

test= pd.read_csv('/content/bank_test.csv',
                       names= ["text","target"])

In [90]:
train.head()

,text,target
0,I APPLIED FOR A CREDIT CARD LAST MONTH BUT I D...,0
1,I OPENED A NEW ACCOUNT IN YOUR BANK BUT WHEN I...,1
2,I APPLIED FOR DEBIT OR CREDIT CARD WHEN I OPEN...,0
3,TILL NOW I DID NOT GET ANY DEBIT OR CREDIT CAR...,0
4,I GAVE AN APPLICATION FOR LINKING UNIQUE NUMBE...,1


In [91]:
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(
#     train,
#     test_size=0.2, # 20% of the data will be used as the test set
#     random_state=42 # Optional: for reproducible results
# )

In [3]:
train.shape

(79, 2)

Number of Intents :

In [4]:
train['target'].value_counts()

target
2    29
1    26
0    24
Name: count, dtype: int64

Preprocessing steps:
1. Target variable --> convert into one hot encoded values
2.

In [5]:
encode_target = oneHot().fit(np.array(train.target).reshape(-1,1))

In [6]:
train_target_encoded = encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded = encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

In [7]:
train_target_encoded

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0

In [8]:
train.text=train.text.map(lambda x : x.lower())
test.text=test.text.map(lambda x: x.lower())

In [9]:
train.head()

,text,target
0,i applied for a credit card last month but i d...,0
1,i opened a new account in your bank but when i...,1
2,i applied for debit or credit card when i open...,0
3,till now i did not get any debit or credit car...,0
4,i gave an application for linking unique numbe...,1


In [10]:
train.text=train.text.map(word_tokenize)
test.text=test.text.map(word_tokenize)

In [11]:
train.text

0     [i, applied, for, a, credit, card, last, month...
1     [i, opened, a, new, account, in, your, bank, b...
2     [i, applied, for, debit, or, credit, card, whe...
3     [till, now, i, did, not, get, any, debit, or, ...
4     [i, gave, an, application, for, linking, uniqu...
                            ...                        
74    [i, want, to, transfer, money, from, my, accou...
75    [i, want, to, transfer, money, from, my, accou...
76    [i, made, a, transaction, with, my, debit, car...
77    [i, made, a, transaction, on, irctc, using, my...
78    [i, applied, for, emi, on, an, online, shoppin...
Name: text, Length: 79, dtype: object

### Remove Stopwords and punctuations

In [12]:
def removeStopWords(strings,stop_word_list):
  sw=[str for str in strings if str not in stop_word_list and  str not in punctuation]
  return sw
stop_words=stopwords.words('english')
train['text']=train.text.map(lambda dataframe : removeStopWords(dataframe,stop_words))
test['text']=test.text.map(lambda dataframe : removeStopWords(dataframe,stop_words))


In [13]:
train['text']

0     [applied, credit, card, last, month, get, one,...
1     [opened, new, account, bank, got, passbook, fo...
2     [applied, debit, credit, card, opened, account...
3     [till, get, debit, credit, card, applied, last...
4     [gave, application, linking, unique, number, b...
                            ...                        
74    [want, transfer, money, account, another, acco...
75    [want, transfer, money, account, another, acco...
76    [made, transaction, debit, card, purchasing, t...
77    [made, transaction, irctc, using, debit, card,...
78    [applied, emi, online, shopping, site, using, ...
Name: text, Length: 79, dtype: object

In [14]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
#Porter stemmer is a class under nltk ,so we need to make an instance of it i.e stemmer and then use

Stemmer= PorterStemmer()

train["text"]= train.text.map(lambda s: [Stemmer.stem(x) for x in s])
train["text"]= train.text.apply(normalize)
test["text"]= test.text.apply(normalize)


In [15]:
train.head()


,text,target
0,appli credit card last month get one till alth...,0
1,open new account bank got passbook found middl...,1
2,appli debit credit card open account bank last...,0
3,till get debit credit card appli last month bank,0
4,gave applic link uniqu number bank account las...,1


In [16]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=1000  #Setting it inilially to 1000
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(train.text)
#fit_on_texts - creates the vocabulary index based on word frequency.


tokenized_train = tokenizer.texts_to_sequences(train.text)
tokenized_test = tokenizer.texts_to_sequences(test.text)

In [17]:
tokenized_train

[[8, 15, 2, 12, 18, 24, 39, 9, 68, 131, 132, 52, 8, 15, 2],
 [53, 31, 1, 11, 40, 54, 41, 133, 42, 69, 70],
 [8, 4, 15, 2, 53, 1, 11, 12, 18, 24, 9],
 [9, 24, 4, 15, 2, 8, 12, 18, 11],
 [16, 10, 17, 25, 7, 11, 1, 12, 18, 9, 55],
 [14, 56, 19, 7, 134, 17, 1, 14, 87, 31, 7],
 [88, 4, 15, 2, 71, 14, 89, 4, 15, 2, 72, 29, 90],
 [73, 89, 4, 15, 2, 88, 4, 15, 2, 71, 72, 29, 90],
 [16, 10, 17, 19, 7, 1, 12, 57, 9, 24, 29, 74],
 [16, 10, 17, 19, 7, 1, 12, 18, 24, 135, 72, 1, 75, 7],
 [3, 30, 43, 91, 92, 32],
 [3, 39, 32, 43, 93, 94, 32],
 [95, 33, 1, 26, 96, 20, 21, 3],
 [136, 20, 1, 137, 27, 138, 97, 76],
 [6, 77, 1],
 [139, 77, 6, 1, 44, 21, 3, 5],
 [11, 1, 30, 140, 77, 76, 5],
 [141, 6, 34, 142, 6, 143, 78, 76, 144, 1],
 [145, 74, 3, 20],
 [6, 146, 98, 147, 148, 99, 3, 20],
 [8, 31, 1, 11, 40, 54, 41, 42, 79, 69, 70],
 [35, 22, 5, 4, 15, 2, 45, 58, 59, 2, 100],
 [5, 4, 2, 46, 5, 20, 45, 58, 59, 2, 100],
 [8, 31, 1, 11, 40, 54, 41, 149, 150, 69, 70, 54],
 [21, 151, 5, 3, 44],
 [14,
  8,
  101

In [18]:
train_data=pad_sequences(tokenized_train,maxlen=20,padding='pre')
test_data=pad_sequences(tokenized_test,maxlen=20,padding='pre')

In [19]:
train_data.shape

(79, 20)

In [20]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data,tokenizer) #This will be the matrix on which the lstm model is applied

trans_matrix_test= transform_matrix(test_data,tokenizer)

In [21]:
trans_matrix_train.shape

(79, 20, 199)

In [22]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.

      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros",
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros",
             kernel_regularizer= l2(l= kernel_reg),
             bias_regularizer= l2(l= bias_reg))(dense_layer_1) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1)

      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model

In [23]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data.
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8.
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps,
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed.
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias.


In [24]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 60) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs.

Epoch 1/60
2/2 [==============================] - 4s 748ms/step - loss: 4.1072 - auc: 0.4953 - val_loss: 3.8962 - val_auc: 0.5664
Epoch 2/60
2/2 [==============================] - 0s 49ms/step - loss: 3.9371 - auc: 0.5985 - val_loss: 3.8653 - val_auc: 0.6387
Epoch 3/60
2/2 [==============================] - 0s 58ms/step - loss: 3.7996 - auc: 0.6708 - val_loss: 3.8351 - val_auc: 0.6855
Epoch 4/60
2/2 [==============================] - 0s 44ms/step - loss: 3.6615 - auc: 0.7584 - val_loss: 3.8057 - val_auc: 0.7070
Epoch 5/60
2/2 [==============================] - 0s 52ms/step - loss: 3.5357 - auc: 0.8274 - val_loss: 3.7763 - val_auc: 0.7402
Epoch 6/60
2/2 [==============================] - 0s 58ms/step - loss: 3.4231 - auc: 0.8791 - val_loss: 3.7470 - val_auc: 0.7637
Epoch 7/60
2/2 [==============================] - 0s 46ms/step - loss: 3.3433 - auc: 0.9106 - val_loss: 3.7182 - val_auc: 0.7773
Epoch 8/60
2/2 [==============================] - 0s 44ms/step - loss: 3.2784 - auc: 0.9332 - va

In [25]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.target, pred_train)) #Print the classification report

3/3 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       1.00      0.92      0.96        26
           2       0.97      1.00      0.98        29

    accuracy                           0.97        79
   macro avg       0.98      0.97      0.97        79
weighted avg       0.98      0.97      0.97        79



In [26]:
import warnings
warnings.filterwarnings('always')
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.target, pred_test)) #Print th

1/1 [==============================] - 0s 30ms/step
              precision    recall  f1-score   support

           0       0.80      0.44      0.57         9
           1       1.00      0.40      0.57         5
           2       0.50      1.00      0.67         7

    accuracy                           0.62        21
   macro avg       0.77      0.61      0.60        21
weighted avg       0.75      0.62      0.60        21

